In [1]:
import sys 
sys.path.append('../')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from NN import network, activation_functions, loss_functions
from NN import ANN


## Simple Run Checks
To identify if there is no code broken

In [2]:
from importlib import reload
reload(network)
I = np.array([[0.05, 0.10]])
W = np.array([[0.15, 0.2], [0.25, 0.3]]).T
h1 = network.Layer(2, 2, 'sigmoid', weights=W, bias=0.35, label="H1")
netH, outH =  h1.feed_forward(I)
print(netH)
print(outH)

[[0.3775 0.3925]]
[[0.59326999 0.59688438]]


In [3]:
from importlib import reload
reload(network)
I = np.array([[0.59326999, 0.59688438]])
W = np.array([[0.4, 0.45], [0.50, 0.55]]).T
o1 = network.Layer(2, 2, 'sigmoid', weights=W, bias=0.6, label="H1")
netO, outO =  o1.feed_forward(I)
print(netO)
print(outO)

[[1.10590597 1.2249214 ]]
[[0.75136507 0.77292847]]


# Convergence Checkings
Based on the sample given by the class teacher

## Manual Calculations

In [4]:
from importlib import reload
reload(network)
model = network.NN(loss='smd')
X = np.array([[0.05, 0.10]])

Wh = np.array([[0.15, 0.2], [0.25, 0.3]]).T
h = network.Layer(2, 2, 'sigmoid', weights=Wh, bias=0.35, label="H1")

Wo = np.array([[0.4, 0.45], [0.50, 0.55]]).T
o = network.Layer(2, 2, 'sigmoid', weights=Wo, bias=0.6, label="H1")
model.add_layer(h)
model.add_layer(o)
model.show_weights()

Y, aY = model.feed_forward(X)
Y_, aY_ = ([1.10590597, 1.2249214 ], [0.75136507, 0.77292847])
print("Y\n",Y_)
print("\na(Y)\n",aY_)

assert((Y - Y_).sum() < np.finfo(np.float32).eps)
assert((aY - aY_).sum() < np.finfo(np.float32).eps)


Y
 [1.10590597, 1.2249214]

a(Y)
 [0.75136507, 0.77292847]


### Working on the back propagation

In [5]:
from importlib import reload
reload(network)
model.show_weights()
Y = np.array([0.01, 0.99])
Etotal = loss_functions.smd(aY, Y)
print(Etotal)
assert((Etotal - 0.2983711) < np.finfo(np.float32).eps)

0.2983711087600027


### Emulating the weights update for the layer O 

In [6]:
reload(loss_functions)
reload(activation_functions)
lr = 0.5
#Done - Partial
dEo_dw = loss_functions.smd_derivative_chain(outO, Y)
print(dEo_dw)

# Done
dOuto_Dneto = activation_functions.sigmoid_derivative_chain(outO)
print('dout',dOuto_Dneto)

# Done - Self.input
dNeto  = outH
print('dnet',dNeto) 
# Done
deltaO  = dEo_dw * dOuto_Dneto
print('delta', deltaO)

dWO = deltaO * outH
print('dw', dWO)
# Done
updateO = Wo - lr * dWO
print('update', updateO)

[[ 0.74136507 -0.21707153]]
dout [[0.1868156  0.17551005]]
dnet [[0.59326999 0.59688438]]
delta [[ 0.13849856 -0.03809824]]
dw [[ 0.08216704 -0.02274024]]
update [[0.35891648 0.51137012]
 [0.40891648 0.56137012]]


In [7]:
#
# Done - Partial
dEo_dOh = deltaO * Wo
dETotal_dOh = dEo_dOh.sum(axis=1)
print (dEo_dOh)
print('dETotal_dOh', dETotal_dOh)

# Done
dOuth_Dneth = activation_functions.sigmoid_derivative_chain(outH)
print('dOuth_Dneth', dOuth_Dneth)


# Done
deltaH = dETotal_dOh * dOuth_Dneth 
print('deltaH', deltaH)

# done
# self.input
dNeth_dw = X
dWh = deltaH * dNeth_dw
print('dWh', dWh)

# Done
updateH = Wh - lr * dWh
print('update', updateH)

[[ 0.05539942 -0.01904912]
 [ 0.06232435 -0.02095403]]
dETotal_dOh [0.03635031 0.04137032]
dOuth_Dneth [[0.24130071 0.24061342]]
deltaH [[0.00877135 0.00995425]]
dWh [[0.00043857 0.00099543]]
update [[0.14978072 0.24950229]
 [0.19978072 0.29950229]]


## Using the implemented Grad calculation for Layers

In [8]:
reload(loss_functions)
reload(activation_functions)
reload(network)
network.DEBUG = False

X = np.array([[0.05, 0.10]], np.float64)

Wh = np.array([[0.15, 0.2], [0.25, 0.3]], np.float64).T
h = network.Layer(2, 2, 'sigmoid', weights=Wh, bias=0.35, label="H1")

Wo = np.array([[0.4, 0.45], [0.50, 0.55]], np.float64).T
o = network.Layer(2, 2, 'sigmoid', weights=Wo, bias=0.6, label="H1")
Y = np.array([0.01, 0.99], np.float64)

for i in range(10000):
    netH, outH = h.feed_forward(X)
    netO, outO = o.feed_forward(outH)

    
    Etotal = loss_functions.smd(outO, Y)
    if (i % 1000) == 0:
        print(i, 'error', Etotal)
    
    dEo_dw = loss_functions.smd_derivative_chain(outO, Y)
    network.dprint(dEo_dw)
    network.dprint ("")
    network.dprint ("==========================================")
    network.dprint ("Back Propagate Layer O")
    network.dprint ("==========================================")
    o.backpropagate(dETotal_dOut=dEo_dw)
    network.dprint ("==========================================")
    network.dprint ("")

    network.dprint ("==========================================")
    network.dprint ("Back Propagate Layer H")
    network.dprint ("==========================================")
    h.backpropagate(output_layer=o)
    network.dprint ("==========================================")

print (outO)

0 error 0.2983711087600027
1000 error 0.001114349453733746
2000 error 0.00044486770391326887
3000 error 0.00025152360241099405
4000 error 0.00016427727363225536
5000 error 0.00011622569943793622
6000 error 8.653979465934606e-05
7000 error 6.676957838285888e-05
8000 error 5.288555201099971e-05
9000 error 4.2742284255545085e-05
[[0.01591419 0.98406371]]


# Using the Fit Methods with the full network calculation

## SMD as the Loss Function

In [9]:
from utils import dataset_helper
reload(loss_functions)
reload(activation_functions)
reload(network)
reload(dataset_helper)
network.DEBUG = False
eps = 0.00001

X = np.array([[0.05, 0.10]], np.float64)
Y = np.array([[0.01, 0.99]], np.float64)

Wh = np.array([[0.15, 0.2], [0.25, 0.3]], np.float64).T
h = network.Layer(2, 2, 'sigmoid', weights=Wh, bias=0.35, label="H1")

Wo = np.array([[0.4, 0.45], [0.50, 0.55]], np.float64).T
o = network.Layer(2, 2, 'sigmoid', weights=Wo, bias=0.6, label="Output")

model = network.NN(loss='smd')
model.add_layer(h)
model.add_layer(o)
model.show_weights()

model.fit(X, Y, max_iter=50000, 
          lr=0.5, epsilon=eps, 
          print_interval=2000)

_, Y_ = model.feed_forward(X)

mae = np.absolute(Y - Y_).mean()
print(Y_, mae)
assert(mae < 0.01)

It: 1 Batch: 1 Epoch 0 Error: 0.29837111 lr: 0.50000000 
It: 2000 Batch: 1 Epoch 1999 Error: 0.00087915 lr: 0.50000000 
It: 4000 Batch: 1 Epoch 3999 Error: 0.00034362 lr: 0.50000000 
It: 6000 Batch: 1 Epoch 5999 Error: 0.00019080 lr: 0.50000000 
It: 8000 Batch: 1 Epoch 7999 Error: 0.00012257 lr: 0.50000000 
It: 10000 Batch: 1 Epoch 9999 Error: 0.00008538 lr: 0.50000000 
It: 12000 Batch: 1 Epoch 11999 Error: 0.00006263 lr: 0.50000000 
It: 14000 Batch: 1 Epoch 13999 Error: 0.00004763 lr: 0.50000000 
It: 16000 Batch: 1 Epoch 15999 Error: 0.00003719 lr: 0.50000000 
It: 18000 Batch: 1 Epoch 17999 Error: 0.00002965 lr: 0.50000000 
It: 20000 Batch: 1 Epoch 19999 Error: 0.00002402 lr: 0.50000000 
It: 22000 Batch: 1 Epoch 21999 Error: 0.00001971 lr: 0.50000000 
It: 24000 Batch: 1 Epoch 23999 Error: 0.00001636 lr: 0.50000000 
It: 26000 Batch: 1 Epoch 25999 Error: 0.00001370 lr: 0.50000000 
It: 28000 Batch: 1 Epoch 27999 Error: 0.00001156 lr: 0.50000000 
Finished 
 It: 29778 Batch: 1 Epoch 29777 

## Cross Entropy 

In [10]:
from utils import dataset_helper
reload(loss_functions)
reload(activation_functions)
reload(network)
reload(dataset_helper)
network.DEBUG = False
eps = 0.117
X = np.array([[0.05, 0.10]], np.float64)
Y = np.array([[0.01, 0.99]], np.float64)

Wh = np.array([[0.15, 0.2], [0.25, 0.3]], np.float64).T
h = network.Layer(2, 2, 'sigmoid', weights=Wh, bias=0.35, label="H1")

Wo = np.array([[0.4, 0.45], [0.50, 0.55]], np.float64).T
o = network.Layer(2, 2, 'sigmoid', weights=Wo, bias=0.6, label="Output")

model = network.NN(loss='cross_entropy')
model.add_layer(h)
model.add_layer(o)
model.show_weights()

model.fit(X, Y, max_iter=50000, 
          lr=0.1, epsilon=eps, 
          print_interval=2000)

_, Y_ = model.feed_forward(X)

mae = np.absolute(Y - Y_).mean()
print(Y_, mae)
assert(mae < 0.01)

It: 1 Batch: 1 Epoch 0 Error: 1.65052853 lr: 0.10000000 
It: 2000 Batch: 1 Epoch 1999 Error: 0.23026661 lr: 0.10000000 
It: 4000 Batch: 1 Epoch 3999 Error: 0.17763656 lr: 0.10000000 
It: 6000 Batch: 1 Epoch 5999 Error: 0.15757797 lr: 0.10000000 
It: 8000 Batch: 1 Epoch 7999 Error: 0.14673802 lr: 0.10000000 
It: 10000 Batch: 1 Epoch 9999 Error: 0.13988631 lr: 0.10000000 
It: 12000 Batch: 1 Epoch 11999 Error: 0.13514499 lr: 0.10000000 
It: 14000 Batch: 1 Epoch 13999 Error: 0.13166350 lr: 0.10000000 
It: 16000 Batch: 1 Epoch 15999 Error: 0.12899745 lr: 0.10000000 
It: 18000 Batch: 1 Epoch 17999 Error: 0.12689090 lr: 0.10000000 
It: 20000 Batch: 1 Epoch 19999 Error: 0.12518550 lr: 0.10000000 
It: 22000 Batch: 1 Epoch 21999 Error: 0.12377782 lr: 0.10000000 
It: 24000 Batch: 1 Epoch 23999 Error: 0.12259734 lr: 0.10000000 
It: 26000 Batch: 1 Epoch 25999 Error: 0.12159422 lr: 0.10000000 
It: 28000 Batch: 1 Epoch 27999 Error: 0.12073225 lr: 0.10000000 
It: 30000 Batch: 1 Epoch 29999 Error: 0.11

In [ ]:
from utils import dataset_helper
reload(loss_functions)
reload(activation_functions)
reload(network)
reload(dataset_helper)
network.DEBUG = False

eps = np.finfo(np.float32).eps

X = np.array([[0.05, 0.10]], np.float64)
Y = np.array([[0.01, 0.99]], np.float64)

h1 = network.Layer(2, 3, 'sigmoid',  bias=0.35, label="H1")
h2 = network.Layer(3, 5, 'sigmoid',  bias=0.35, label="H1")
h3 = network.Layer(5, 3, 'sigmoid',  bias=0.35, label="H1")
o = network.Layer(3, 2, 'sigmoid', bias=0.6, label="Output")

model = network.NN(loss='cross_entropy')
model.add_layer(h1)
model.add_layer(h2)
model.add_layer(h3)
model.add_layer(o)
model.show_weights()

model.fit(X, Y, max_iter=50000, 
          lr=0.1, epsilon=eps, 
          print_interval=2000)

_, Y_ = model.feed_forward(X)

mae = np.absolute(Y - Y_).mean()
print(Y_, mae)
assert(mae < 0.01)

It: 1 Batch: 1 Epoch 0 Error: 1.39996910 lr: 0.10000000 
It: 2000 Batch: 1 Epoch 1999 Error: 0.13694577 lr: 0.10000000 
It: 4000 Batch: 1 Epoch 3999 Error: 0.12443844 lr: 0.10000000 
It: 6000 Batch: 1 Epoch 5999 Error: 0.11978128 lr: 0.10000000 
It: 8000 Batch: 1 Epoch 7999 Error: 0.11737623 lr: 0.10000000 
